James Gardner, March 2022

In [ ]:
# overkill but whatever
from benchmarking import *

In [ ]:
# initialise network
# fixed location for comparing PSDs (noise doesn't depend on it but detector response does, except eq3.6 suggests otherwise, see confusion below)
loc = 'H'
# detector technologies
tecs_2GET = ['aLIGO',
             'A+', 'V+', 'K+',
             'Voyager-CBO', 'Voyager-PMO',
             'ET']
tecs_CE = [f'CE{n}-{Larm}-{opt}' for n in [1, 2] for opt in ['CBO', 'PMO'] for Larm in range(10, 50, 10)]

# omni-network
network_spec = [f'{tec}_{loc}' for tec in tecs_2GET + tecs_CE] # using list concatenation (+) 
locs = [loc for _ in network_spec]
net = network.Network(network_spec)

# frequency range
f = np.geomspace(1e0, 4e3, 100)

# choose the desired waveform 
# wf_model_name, wf_other_var_dic = 'tf2', None
# net.set_wf_vars(wf_model_name=wf_model_name)

# injection parameters, for GW150914
inj_params = {
    'Mc':    30.9,
    'eta':   0.247,
    'chi1z': 0,
    'chi2z': 0,
    'DL':    475,
    'tc':    0,
    'phic':  0,
    'iota':  PI/4,
    'ra':    PI/4,
    'dec':   PI/4,
    'psi':   PI/4,
    'gmst0': 0,
}

# derivative settings
deriv_symbs_string = 'Mc eta DL tc phic iota ra dec psi'
conv_cos = ('iota', 'dec')
conv_log = ('Mc', 'DL')
# network settings
use_rot = 0
only_net = 1

net.set_net_vars(
    f=f, inj_params=inj_params,
    deriv_symbs_string=deriv_symbs_string,
    conv_cos=conv_cos, conv_log=conv_log,
    use_rot=use_rot
    )
print('Network initialised')

In [ ]:
# just psd required
net.setup_ant_pat_lpf_psds()

In [ ]:
# plot sensitivity curves together, a la Fig 5 in Borhanian 2021
colours_2GET = [*[(1, 0, 1, i) for i in np.arange(0.25, 1.25, 0.25)], (0, 1, 0, 0.5), (0, 1, 0, 1), (1, 0.5, 0, 1)]
colours_CE = [(*rgb, Larm/40) for rgb in ((1, 0, 0), (0, 0, 1)) for Larm in range(10, 50, 10)]

plt.rcParams.update({'font.size': 16})
fig, axs = plt.subplots(2, 3, figsize=(16, 8), gridspec_kw={'wspace':0.05, 'hspace':0.1})
for i, detector in enumerate(net.detectors):
    # detector signal response is sky-position and polarisation dependent (in honours I assumed perfect alignment)
#     axs[1].loglog(detector.f, np.abs(detector.hf))
    # confused, formula for PSD looks like noise spectrum with no trace of signal transfer function
    # SNR is calculated as H/Sn, which suggests that Sn is the spectrum of noise
    # however, the units and scale are correct for the ASD of NSR in the plot
    if detector.tec in tecs_2GET:
        tec_type = 0
        color=colours_2GET[i]
    # else, is in CE
    else:
        tec_type = int(detector.tec[2])
        color=colours_CE[tecs_CE.index(detector.tec) % 8]
                
    axs[0,tec_type].loglog(detector.f, np.sqrt(detector.psd), label=detector.tec, color=color)
    axs[1,tec_type].loglog(detector.f, np.sqrt(detector.psd), label=detector.tec, color=color)

# top row
for i, ax in enumerate(axs[0]):
    ax.legend(fontsize=14, ncol=2, columnspacing=1, handlelength=1)
    ax.set(xlim=(f[0], f[-1]), ylim=(1e-25, 1e-15))
    ax.set_xticklabels([])
    if i > 0:
        ax.set_yticklabels([]) 
    
# bottom row
for i, ax in enumerate(axs[1]):
    # frequency range truncated to range specified in PSD
    ax.set(xlabel='frequency, f / Hz', xlim=(f[0], f[-1]), ylim=(2e-25, 1e-23))
    if i > 0:
        ax.set_yticklabels([])
    
axs[0,0].set_ylabel(r'sensitivity / $\mathrm{Hz}^{-1/2}$')
axs[1,0].set_ylabel(r'sensitivity / $\mathrm{Hz}^{-1/2}$')

plt.show()
fig.savefig('plots/sensitivity_curves.pdf')
plt.close(fig)

In [ ]:
# to-do: resolve this confusion about noise, signal, and sensitivity